In [1]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(tokenizer_file="tokenizer.json", 
                                    bos_token="<s>", eos_token="</s>", unk_token="<unk>", pad_token="<pad>")

In [2]:
from transformers import BartForConditionalGeneration
model = BartForConditionalGeneration.from_pretrained('models/kana-kanji/checkpoint-290000')

In [3]:
from datasets import load_dataset, DatasetDict
dataset_dict = DatasetDict.load_from_disk('dataset_all')

In [4]:
import torch
def make_predictions_and_refereces(dataset):
    input_ids = torch.IntTensor(dataset['input_ids'])
    generated_ids = model.generate(input_ids, num_beams=5, max_length=1024, early_stopping=True)
    generated_texts_tokenized = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    target_texts = [' '.join(tokenizer.tokenize(text)) for text in dataset['plain_text']]
    return generated_texts_tokenized, target_texts

In [5]:
predictions, references = make_predictions_and_refereces(dataset_dict['test'][:5])
for p, r in zip(predictions, references):
    print(f"reference:  {r}")
    print(f"predictions: {p}\n")

reference:  実は 図 書 館 で ＤＮＡ について の 本 を 借 り た んですよ
predictions: 実は 図 書 館 で ＤＮＡ について の 本 を 借 り た んですよ

reference:  で<sp> 次に デ ジ タ ル ポ ート フォ リ オ ア セ ス メント の方 ですが <F>ま</F> デ ジ タ ル ポ ート フォ リ オ の 構 築 パー ソ ナ ル ポ ート フォ リ オ への 解釈 意味 付け による 学習 者の 自 己 評価
predictions: で<sp> 次に デ ジ タ ル ポ ート フォ リ オ ア セ ス メント の方 ですが <F>ま</F> デ ジ テ ラ ル 法 と フォ リ ク パー ソ ナ ル ポ ント フォ リ エ の 解釈 意味 付け による 学習 者の 自 己 評価

reference:  など を 含 め ると
predictions: など を 含 め ると

reference:  <F>あのー</F><sp> じゃ 今日 は<F>あの</F> ちょっと 家 に 帰 れない から ど っか 泊 ま ろう <F>んー</F> ホ テ ル 泊 ま んな きゃ いけない やっぱ お金 お金 掛 から なきゃ いけない <sp> な っていう とか
predictions: <F>あのー</F><sp> じゃ 今日 は<F>あの</F> ちょっと 家 に 帰 れない から ど っか 泊 ま ろう <F>んー</F> ホ テ ル 止 ま んな きゃ いけない やっぱ お金 掛 から なきゃ いけない <sp> な っていう とか

reference:  <F>と</F> 調査 結果 を まとめ ます
predictions: <F>と</F> 調査 結果 を まとめ ます



In [9]:
import evaluate
metric_cer = evaluate.load("cer")
metric_wer = evaluate.load("wer")
metric_bleu = evaluate.load("bleu")
metric_rouge = evaluate.load("rouge")
# metrics = evaluate.combine(["wer", "bleu", "rouge"])

In [23]:
import tqdm

In [29]:
num_data = len(dataset_dict['test'])
batch_size = 32
num_batches = (num_data + batch_size - 1) // batch_size
for i in tqdm.tqdm(range(num_batches)):
    start = i * batch_size
    end = min((i + 1) * batch_size, num_data)
    predictions, references = make_predictions_and_refereces(dataset_dict['test'][start:end])
    predictions_without_whitespace = [p.replace(' ', '') for p in predictions]
    references_without_whitespace = [r.replace(' ', '') for r in references]
    metric_cer.add_batch(predictions=predictions_without_whitespace, 
                            references=references_without_whitespace)
    metric_wer.add_batch(predictions=predictions, references=references)
    metric_bleu.add_batch(predictions=predictions, references=references)
    metric_rouge.add_batch(predictions=predictions, references=references)
    # metrics.add_batch(predictions=predictions, references=references)
    

100%|██████████| 32/32 [06:06<00:00, 11.45s/it]


In [30]:
cer_score = metric_cer.compute()
wer_score = metric_wer.compute()
bleu_score = metric_bleu.compute()
rouge_score = metric_rouge.compute()

In [31]:
cer_score, wer_score, bleu_score, rouge_score

(0.014340080525067564,
 0.037589291906512244,
 {'bleu': 0.9629690911764732,
  'precisions': [0.9817547399840248,
   0.9705095010312897,
   0.9598222466556716,
   0.9494925315970892],
  'brevity_penalty': 0.9975646636169296,
  'length_ratio': 0.9975676242398825,
  'translation_length': 47574,
  'reference_length': 47690},
 {'rouge1': 0.7198958333333334,
  'rouge2': 0.5961681818181819,
  'rougeL': 0.7193541666666666,
  'rougeLsum': 0.7200416666666667})